In [1]:
#import sys
#import subprocess

#subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "numpy", "statsmodels", "matplotlib", "scipy", "openpyxl"])

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.discrete_model import Probit
import matplotlib.pyplot as plt
import scipy.stats as stats


# Data Preparation

1) Load the cleaning function we created last time
2) Use the function to prepare the SPX and NVDA data

In [3]:
# you can use this cleaning function we created last time
def data_cleaning(benchm, asset):

    benchm["Date"] = pd.to_datetime(benchm["Date"])
    benchm = benchm.sort_values(by='Date', ascending=True) 

    asset["Date"] = pd.to_datetime(asset["Date"])
    asset = asset.sort_values(by='Date', ascending=True) 

    comb = benchm.merge(asset,on="Date", how="left")

    comb['return_benchmark'] = comb.iloc[:, 1] / comb.iloc[:, 1].shift(1)-1
    comb['return_asset'] = comb.iloc[:, 2] / comb.iloc[:, 2].shift(1)-1
    #comb['logreturn_benchmark'] = np.log(comb.iloc[:, 1] / comb.iloc[:, 1].shift(1))
    #comb['logreturn_asset'] = np.log(comb.iloc[:, 2] / comb.iloc[:, 2].shift(1))
    comb = comb.dropna()

    return comb

In [ ]:
spx_raw = pd.read_excel('SPX.xlsx')
nvda_raw = pd.read_excel('NVDA.xlsx')

data = data_cleaning(spx_raw, nvda_raw)

# make column labels explicit for clarity
data = data.rename(columns={
    data.columns[1]: 'SPX',
    data.columns[2]: 'NVDA',
    'return_benchmark': 'SPX_return',
    'return_asset': 'NVDA_return'
})

data.head()


3) Load the Fama-French 5-factor (FF5) time-series data
4) Format the dates to datetime format
5) Merge the SPX/NVDA data and the FF5 data using the SPX/NVDA dataframe as the leading dataframe
6) Compute excess returns for NVDA by subtracting the risk-free rate (column "RF" in FF5) from simple NVDA returns

In [20]:
FF5 = pd.read_excel('F-F_Research_Data_5_Factors_2x3_daily.xlsx')
FF5['Date'] = pd.to_datetime(FF5['Date'], format='%Y%m%d')

factor_cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF']
FF5[factor_cols] = FF5[factor_cols].apply(lambda col: pd.to_numeric(col, errors='coerce')) / 100

data = data.merge(FF5, how='left', on='Date')
data = data.dropna(subset=['NVDA_return'] + factor_cols).reset_index(drop=True)

data['excess_ret_asset'] = data['NVDA_return'] - data['RF']

data[['Date', 'NVDA_return', 'RF', 'excess_ret_asset']].head()


        Date  Mkt-RF   SMB   HML   RMW   CMA    RF
0 1963-07-01   -0.67  0.00 -0.34 -0.01  0.16  0.01
1 1963-07-02    0.79 -0.26  0.26 -0.07 -0.20  0.01
2 1963-07-03    0.63 -0.17 -0.09  0.18 -0.34  0.01
3 1963-07-05    0.40  0.08 -0.27  0.09 -0.34  0.01
4 1963-07-08   -0.63  0.04 -0.18 -0.29  0.14  0.01
        Date      SPX    NVDA  return_benchmark  return_asset
1 2023-01-04  3852.97  14.749          0.007539      0.030318
2 2023-01-05  3808.10  14.265         -0.011646     -0.032816
3 2023-01-06  3895.08  14.859          0.022841      0.041640
4 2023-01-09  3892.09  15.628         -0.000768      0.051753
5 2023-01-10  3919.25  15.909          0.006978      0.017981


# Multivariate Regression

1) Regress NVDA excess returns on a constant and all 5 Fama-French factors ("Mkt-RF", "SMB", "HML", "RMW", "CMA")

In [26]:
Y = data['excess_ret_asset']
X = data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       excess_ret_asset   R-squared:                       0.534
Model:                            OLS   Adj. R-squared:                  0.531
Method:                 Least Squares   F-statistic:                     146.7
Date:                Sat, 11 Oct 2025   Prob (F-statistic):          1.40e-103
Time:                        10:28:35   Log-Likelihood:                 1531.7
No. Observations:                 645   AIC:                            -3051.
Df Residuals:                     639   BIC:                            -3025.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0174      0.001    -19.431      0.000      -0.019      -0.016
Mkt-RF         0.0203      0.001     20.766      0.000       0.018       0.022
SMB           -0.0051      0.002     -3.334      0.001      -0.008      -0.002
HML           -0.0137      0.001     -9.209      0.000      -0.017      -0.011
RMW            0.0052      0.002      2.529      0.012       0.001       0.009
CMA            0.0032      0.002      1.639      0.102      -0.001       0.007
==============================================================================
Omnibus:                      273.012   Durbin-Watson:                   2.124
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4179.119
Skew:                           1.464   Prob(JB):                         0.00
Kurtosis:                      15.122   Cond. No.                         2.96
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Deutung der Tabelle:

Also schaue in der coffezienze tabelle, was kleiner und das will ich mir anschauen alsi umsokleiner P>T dings umso wischtiger ist das. Z.B. CMA ist zu hoch deswegen sit das scheiß egal weil es 0.102 ist das heiußt das einfach zu hoch

Dann interepretation was mache ich wenn als bsp mein Market RF wenn das um 1 hoch geht dann geht NVDA um 0.0203 hoch also um den wert der da angegeben wird bei coef. bzw wenn iche s weiter eherhöe dann gehts weiter hoch.

Prp: durch slides gehen und checken wo die alle zu finden sind in der tabelle so die connections machen. Glaube da wird dann alles abgefragt also z.B. Skew was das was mcht das etc was R2 und so also die tabelle verstehen und wissen wo alles steht





Hier noch die extra scahen

In [ ]:
factor_loadings = pd.DataFrame({
    'coef': model.params,
    't_stat': model.tvalues,
    'p_value': model.pvalues
})
factor_loadings


# FED Probit model

1) Load the dataset containing the difference between the 3-month and 10-year Treasury yield (t10y3m)
2) Ensure the dates are in datetime format and create a new column that contains the first date of the month to use it as a month indicator
3) Group data by that month indicator taking the monthly mean of the yield spread

In [32]:
t10y3m = pd.read_excel("T10Y3M.xlsx")
t10y3m["observation_date"] = pd.to_datetime(t10y3m["observation_date"])
t10y3m["month"] = t10y3m["observation_date"].dt.to_period("M").dt.to_timestamp()
t10y3m = t10y3m.groupby("month",as_index=False)["T10Y3M"].mean()
t10y3m

,month,T10Y3M
0,1982-01-01,1.676000
1,1982-02-01,0.146111
2,1982-03-01,0.546522
3,1982-04-01,0.527143
4,1982-05-01,0.908500
...,...,...
521,2025-06-01,-0.037500
522,2025-07-01,-0.020455
523,2025-08-01,-0.040000
524,2025-09-01,0.054762


4) Load the dataset with monthly recession indicators and ensure the date column is in datetime format

In [37]:
usrce = pd.read_excel('USREC.xlsx')
usrce['observation_date'] = pd.to_datetime(usrce['observation_date'])
usrce['month'] = usrce['observation_date'].dt.to_period('M').dt.to_timestamp()
usrce = usrce.groupby('month', as_index=False)['USREC'].mean()
usrce.head()


5) Merge the two dataframes using on the month indicator the yield spread as the leading dataframe
6) Create a new column that gives the 12 months ahead recession indicator using the .shift() method
7) Drop NaN observations

In [39]:
fed_rec = t10y3m.merge(usrce, how='left', on='month')
fed_rec['USREC_12m'] = fed_rec['USREC'].shift(-12)
fed_rec = fed_rec.dropna(subset=['USREC_12m']).reset_index(drop=True)
fed_rec.head()


,month,T10Y3M,USREC
0,1982-01-01,1.676000,1.0
1,1982-02-01,0.146111,1.0
2,1982-03-01,0.546522,1.0
3,1982-04-01,0.527143,1.0
4,1982-05-01,0.908500,1.0
5,1982-06-01,1.220909,1.0
6,1982-07-01,2.082857,1.0
7,1982-08-01,4.057727,1.0
8,1982-09-01,4.146190,1.0
9,1982-10-01,2.939000,1.0


8) Estimate a probit model with an intercept, the yield spread as an independent variable and 12 month ahead recession indicator as the dependent variable using data until December 2009

In [ ]:
est_sample = fed_rec[fed_rec['month'] <= '2009-12-31']
X_est = sm.add_constant(est_sample['T10Y3M'])
y_est = est_sample['USREC_12m']

probit_model = Probit(y_est, X_est).fit(disp=False)
probit_model.summary()


9) Based on this model, predict the probability of a recession for each month in the entire dataset and plot that probability over time 

In [ ]:
X_all = sm.add_constant(fed_rec['T10Y3M'])
fed_rec['recession_prob'] = probit_model.predict(X_all)

fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(fed_rec['month'], fed_rec['recession_prob'], label='Predicted probability')
ax.fill_between(
    fed_rec['month'],
    0,
    1,
    where=fed_rec['USREC'] > 0.5,
    color='red',
    alpha=0.1,
    label='Recession (current)'
)
ax.set_xlabel('Month')
ax.set_ylabel('Probability of recession')
ax.set_title('Predicted U.S. Recession Probability (12 Months Ahead)')
ax.set_ylim(0, 1)
ax.legend(loc='upper left')
fig.tight_layout()
